# Exercise 1: Optimize lepton selection

* First, print the distributions of the relevant variables for *all* the Monte Carlo samples (i.e. all the *channels* of the $Z$-boson decay to be studied). Which variables are these? Give sensible ranges to include all the events in the samples (both MC and OPAL data) 
* Do the same for **one** of the OPAL data samples (your lab assistant will decide which one you choose).
* Describe the results.
* Optimize the object selection by applying cuts. Make a strategy on how to proceed to find the optimal selection. which information do you need? in thin the
* Determine the efficiency and the amount of background for each $Z$ decay channel. Use the simulated events $e^+e^-$, $\mu^+\mu^-$, $\tau^+\tau^-$ and hadrons ($qq$). Represent the result in a matrix form and think carefully about how you have to correct the measured rates. Don't forget to calculate the errors!
* How do we estimate the statistical fluctuations per bin?

### Download and import libraries
Comment in the following two lines in case some of the libraries cannot be imported. Please restart the kernel after download+upgrade has successfully finished

In [ ]:
### Download libraries
#%pip install uproot
#%pip install awkward
#%pip install mplhep
#%pip install numpy
#%pip install matplotlib
#%pip install scipy

### Upgrade libraries to latest version
#%pip install uproot awkward mplhep numpy matplotlib scipy --upgrade

In [ ]:
import uproot
import awkward as ak
import mplhep
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches
import pandas as pd
from scipy.optimize import curve_fit

%matplotlib inline
plt.rcParams.update({'font.size': 11})

## Import Monte Carlo Samples and define Cuts
First, we will open the Monte Carlo samples using `uproot`.

The datasets are converted to a pandas data frame. Available keys are:

| Variable name | Description |
| --- | --- | 
| ``run`` | Run number |
| ``event`` | Event number |
| ``Ncharged`` | Number of charged tracks |
| ``Pcharged`` | Total scalar sum of track momenta |
| ``E_ecal`` | Total energy measured in the electromagnetic calorimeter |
| ``E_hcal`` | Total energy measured in the hadronic calorimete |
| ``E_lep`` | LEP beam energy (=$\sqrt{s}/2$) |
| ``cos_thru`` | cosine of the polar angle between beam axis and thrust axis |
| ``cos_thet`` | cosine of the polar angle between incoming positron and outgoing positive particle |

If ``cos_thru`` or ``cos_thet`` are undefined, the value ``999`` is used to idicate that. This is replaced by ``NaN`` to ensure correct behaviour in calculations.

Furthermore, we reject values larger $100\,\mathrm{MeV}$ for ``Pcharged`` as unphysical, replacing them also with ``NaN``. This is necessary, since outliers $>10000\,\mathrm{MeV}$ disturb the plotting routine and averaging procedures.

In [ ]:
path_mc = 'opal_data/mc/'   # monte-carlo simulations

### Open the file introducing file path
ee = uproot.open(path_mc+'ee.root')
mm = uproot.open(path_mc+'mm.root')
tt = uproot.open(path_mc+'tt.root')
qq = uproot.open(path_mc+'qq.root')
ttree_name = 'myTTree'

### Get list of 'branches' of the TTree (i.e. list of variable names)
keys = ee[ttree_name].keys()

## Load branches
ee_branches = ee[ttree_name].arrays()
mm_branches = mm[ttree_name].arrays()
tt_branches = tt[ttree_name].arrays()
qq_branches = qq[ttree_name].arrays()

## Convert to pandas data frame
simulated_electron = ak.to_pandas(ee_branches)
simulated_muon = ak.to_pandas(mm_branches)
simulated_tau = ak.to_pandas(tt_branches)
simulated_quark = ak.to_pandas(qq_branches)

## Efficiency correction factor (between nominal 100,000 events and the actual number of events)
eff_corr_electons = len(simulated_electron) / 1e5
eff_corr_muons = len(simulated_muon) / 1e5
eff_corr_taus = len(simulated_tau) / 1e5
eff_corr_quarks = len(simulated_quark) / 1e5

## Delete photon-photon entries
simulated_electron = simulated_electron[simulated_electron['Pcharged']!=0]
simulated_muon = simulated_muon[simulated_muon['Pcharged']!=0]
simulated_tau = simulated_tau[simulated_tau['Pcharged']!=0]
simulated_quark = simulated_quark[simulated_quark['Pcharged']!=0]

## Iterable arrays of different event types
simulated_datasets = {'electron': simulated_electron, 'muon': simulated_muon, 'tau': simulated_tau, 'quark': simulated_quark}
efficiency_corrections = {'electron': eff_corr_electons, 'muon': eff_corr_muons, 'tau': eff_corr_taus, 'quark': eff_corr_quarks}

## Use correct NaN specifier for undefined values
for particle, dataset in simulated_datasets.items():
    dataset['cos_thru'].replace(999, np.nan, inplace=True)
    dataset['cos_thet'].replace(999, np.nan, inplace=True)
    dataset['Pcharged'].mask(dataset['Pcharged']>100, inplace=True)

### Visualization of the dataset

We can now look at the ditribution of values for the physical quantities of the dataset.

For that, we plot histograms of all keys in the data frame except ``run`` and ``event`` number and the LEP beam energy, which for those samples was always constant.

In [ ]:
# Relevant variables within the sample
relevant_variables = ['Ncharged', 'Pcharged', 'E_ecal', 'E_hcal', 'cos_thru', 'cos_thet']

# Show histograms for each dataset with pandas
if False:   # set True to print all the histograms
    for particle, dataset in simulated_datasets.items():
        print(particle)
        diagram_list = dataset.hist(bins=50, column=relevant_variables)
        plt.tight_layout()
        plt.show()

For a better visualization of the parameter space, we can also plot each set of two variables in a scatter plot:

In [ ]:
## get the default matplotlib color cycle as indexable array (needed to later prepare the legend)
prop_cycle = plt.rcParams['axes.prop_cycle']
default_colors = prop_cycle.by_key()['color']


## plot all combinations of interesting variables except for the angles
if False:
    for i in range(4):   # variable to be plotted on the x axis
        for j in range(i+1, 4):   # variable to be plotted on the y axis

            # plot faint datapoints
            for k, (particle, dataset) in enumerate(simulated_datasets.items()):
                plt.plot(dataset[relevant_variables[i]], dataset[relevant_variables[j]], '.', ms=0.2, alpha=0.1)   # actual data
                plt.plot([], [], '.', color=default_colors[k], ms=5, alpha=1, label=particle)   # better visible dots for legend

            plt.xlabel(relevant_variables[i])
            plt.ylabel(relevant_variables[j])
            plt.grid()
            plt.legend()
            plt.show()

### Declaring cuts

Separating the quark contributions is easily possible by their large number of charged traces:

We declare all events with `Ncharged >= 10` to be quark events, all with `Ncharged <=6` to be non-quark events.

For the leptons we could see the largest spread in the `Pcharged`-`E_ecal` 2D plot, there we choose regions differenciate the three types.

In [ ]:
## declare cuts
Nmin_quarks = 10
Nmax_leptons = 6
Emin_ele = 80
Emax_muons = 20
Pmin_muons = 75
Emax_taus = 60
Pmin_taus = 5
Pmax_taus = 55



## plot histogram
histogram_data = [simulated_electron['Ncharged'],simulated_muon['Ncharged'],simulated_tau['Ncharged'],simulated_quark['Ncharged']]
plt.hist(histogram_data, histtype='step', bins=48, color=default_colors[:4])   # outline
plt.hist(histogram_data, histtype='stepfilled', bins=48, alpha=0.25, color=default_colors[:4])   # filling
plt.plot([Nmin_quarks, Nmin_quarks], [0, 1e4], c='m', ls=':', lw=2.5)  # plot separating lines
plt.plot([Nmax_leptons+1, Nmax_leptons+1], [0, 1e4], c='m', ls=':', lw=2.5)  # plot separating lines
plt.plot()
plt.xlim(4,15)
plt.ylim(0,2500)
#plt.savefig('figures/selection_histogram.pdf')
plt.show()


## scatter plot
for k, (particle, dataset) in enumerate(simulated_datasets.items()):
    if particle != 'quark':
        plt.plot(dataset['Pcharged'], dataset['E_ecal'], '.', ms=0.2, alpha=0.1)   # actual data
        plt.plot([], [], '.', color=default_colors[k], ms=5, alpha=1, label=particle)   # better visible dots for legend
    
    ## visualize cuts by plotting separating lines
    plt.plot([0, 100], [Emin_ele, Emin_ele], c='b', ls='--', lw=1.5)
    plt.plot([Pmin_muons, Pmin_muons], [0, Emax_muons], c='y', ls='--', lw=1.5)
    plt.plot([Pmin_muons, 100], [Emax_muons, Emax_muons], c='y', ls='--', lw=1.5)
    plt.plot([Pmin_taus, Pmax_taus], [Emax_taus, Emax_taus], c='g', ls='--', lw=1.5)
    plt.plot([Pmin_taus, Pmin_taus], [0, Emax_taus], c='g', ls='--', lw=1.5)
    plt.plot([Pmax_taus, Pmax_taus], [0, Emax_taus], c='g', ls='--', lw=1.5)

plt.xlabel('Pcharged')
plt.ylabel('E_ecal')
plt.grid()
plt.legend()
plt.xlim(0, 100)
plt.ylim(0,120)
#plt.savefig('figures/selection_scatter.pdf')
plt.show()


## define functions to produce the cutting mask on any dataset

def electron_mask(dataset):
    return np.all((dataset['Ncharged'] <= Nmax_leptons, dataset['E_ecal'] >= Emin_ele), axis=0)

def muon_mask(dataset):
    return np.all((dataset['Ncharged'] <= Nmax_leptons, dataset['E_ecal'] <= Emax_muons, dataset['Pcharged'] >= Pmin_muons), axis=0)

def tau_mask(dataset):
    return np.all((dataset['Ncharged'] <= Nmax_leptons, dataset['E_ecal'] <= Emax_taus,
                   dataset['Pcharged'] >= Pmin_taus, dataset['Pcharged'] <= Pmax_taus), axis=0)

def quark_mask(dataset):
    return dataset['Ncharged'] >= Nmin_quarks

## Iterable array of different selection types
selection_masks = {'electron': electron_mask, 'muon': muon_mask, 'tau': tau_mask, 'quark': quark_mask}

Applying those cuts to the monte carlo data gives us the selection efficiencies; via matrix inversion the correcting factors for the measured data can be calculated.

In [ ]:
## calculate and save selection rates
selection_rates = np.zeros((4,4))
selection_errors = np.zeros((4,4))

for i, (particle, dataset) in enumerate(simulated_datasets.items()):
    for j, (rule_particle, selection_rule) in enumerate(selection_masks.items()):
        mask = selection_rule(dataset)
        selection_rates[i][j] = sum(mask)/len(mask) * efficiency_corrections[particle]
        selection_errors[i][j] = np.sqrt(sum(mask))/len(mask) * efficiency_corrections[particle]
        print(f"Out of {len(mask)} " + particle + f"-events, {np.sum(mask)} passed the " + rule_particle + "-selection mask")
        
print('Selection rates: \n', selection_rates)    

In [ ]:
# Define gaussian function to fit to the toy distributions:
def gauss(x, A, mu, sigma):
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))


### Number of toy experiments to be done
ntoy = 100000

### Create numpy matrix of list to append elements of inverted toy matrices
inverse_toys = np.empty((4,4,ntoy))

# Create toy efficiency matrix out of gaussian-distributed random values
for i in range(ntoy):
    toy_matrix = np.random.normal(selection_rates, selection_errors, size=(4,4))
    
    ### Invert toy matrix
    inverse_toys[:,:,i] = np.linalg.inv(toy_matrix)


inverse_errors = np.zeros((4,4))
inverse_means = np.zeros((4,4))


fig, axes = plt.subplots(4, 4, figsize=(20, 10),dpi=80)


# Fill histograms for each inverted matrix coefficient:
for j in range(4):
    for k in range(4):
        
        ## Calculate empirical mean and standard deviation directly
        mean = np.mean(inverse_toys[j,k,:])
        std = np.sqrt(np.sum((inverse_toys[j,k,:] - mean)**2 )/ (ntoy-1))
        
        # Diagonal and off-diagonal terms have different histogram ranges
        hbins, hedges, _ = axes[j][k].hist(inverse_toys[j,k,:], bins=300, histtype='step', linewidth=2,
                                           range=(mean-5*std, mean+5*std),  label=f'toyhist{j}{k}')
        axes[j][k].legend()

        # Get the fitted curve
        h_mid = 0.5*(hedges[1:] + hedges[:-1]) # Calculate midpoints for the fit
        coeffs, _ = curve_fit(gauss, h_mid, hbins, maxfev=10000, p0=(1, mean, std))
        
        axes[j][k].plot(h_mid, gauss(h_mid, *coeffs), label=f'Fit{j}{k}')

        inverse_means[j,k] = coeffs[1]
        inverse_errors[j,k] = abs(coeffs[2])

print(f"Errors for the inverse matrix:\n{inverse_errors}")

## Import OPAL data

In [ ]:
### Open opal data and read into pandas
path_OPAL = 'opal_data/data/'   # actual data  simulations
OPAL = uproot.open(path_OPAL+'daten_2.root')
OPAL_branches = OPAL[ttree_name].arrays()   # load branches
OPAL_data = ak.to_pandas(OPAL_branches)   # convert to pandas data frame
OPAL_data = OPAL_data[OPAL_data['Pcharged']!=0]   # remove photon-photon contributions

## Use correct NaN specifier for undefined values
OPAL_data['cos_thru'].replace(999, np.nan, inplace=True)
OPAL_data['cos_thet'].replace(999, np.nan, inplace=True)
OPAL_data['Pcharged'].mask(OPAL_data['Pcharged']>100, inplace=True)

## extract different event types
measured_electron = OPAL_data[electron_mask(OPAL_data)]
measured_muon = OPAL_data[muon_mask(OPAL_data)]
measured_tau = OPAL_data[tau_mask(OPAL_data)]
measured_quark = OPAL_data[quark_mask(OPAL_data)]

## Iterable array of different event types
measured_datasets = {'electron': measured_electron, 'muon': measured_muon, 'tau': measured_tau, 'quark': measured_quark}

In [ ]:
OPAL_data.hist(bins=52)
plt.show()

# Exercise 2: Separate $t$- and $s$-channel contributions

Only Feynman diagrams contributing to the production of $Z$ boson are to be considered for the measurements. The **electron** Monte Carlo sample incorporate contributions from $t$- and $s$-channels.
* Select/correct contributions producing $Z$ boson decays. (Hint: Which role does the $\cos(\theta)$ distribution play in separating $t$- and $s$-channels?)

In [ ]:
## define fit function
def func_t_s_separation(cos_theta, A, B):
    return A * (1 + cos_theta**2) + B * (1 - cos_theta)**(-2)

def func_s_channel(cos_theta, A, B):
    return A * (1 + cos_theta**2)

def func_rest(cos_theta, A, B):
    return B * (1 - cos_theta)**(-2)


# this function performs the separation via a curve fit
def perform_t_s_separation(the_input_dataset, make_plot=True, bins=100, limit=0.85):
    # create histogram from given dataset
    electron_cos_theta, bin_edges, _ = plt.hist(the_input_dataset['cos_thet'],bins=bins, range=(-1.0,1.0),  histtype='stepfilled', label='Pcharged')
    
    # calculate bin centers
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    if make_plot:
        plt.tight_layout()
        plt.clf()   #plt.show() if you also want to view the histogram
    
    # mask bins in desired region for curve fit
    curve_fit_mask = np.logical_and(bin_edges[:-1] >= -limit, bin_edges[1:] <= limit)
    
    # calculate curve fit in desired region
    t_s_popt, t_s_pcov = curve_fit(func_t_s_separation, bin_centers[curve_fit_mask], electron_cos_theta[curve_fit_mask], sigma=np.sqrt(electron_cos_theta[curve_fit_mask]), maxfev=int(1e6), p0=[2, .2])
    t_s_perr = np.sqrt(np.diag(t_s_pcov))
    
    thetas = np.linspace(-limit, limit, 10000)
    # plot results if desired
    if make_plot:
        textstr = '\n'.join((
            r'$P_\mathrm{meas} = a \cdot P_\mathrm{nom}+b$',
            r'$A=(%.3f \pm %.3f)$' % (t_s_popt[0], t_s_perr[0]),
            r'$B=(%.2f \pm %.2f)$' % (t_s_popt[1], t_s_perr[1])))


        props = dict(alpha=1, facecolor='w')

        plt.errorbar(bin_centers, electron_cos_theta, yerr=np.sqrt(electron_cos_theta), fmt='.', ms=3, ecolor='r', elinewidth=1)
        plt.plot(thetas, func_t_s_separation(thetas, *t_s_popt),lw=0.7, c='purple', label='entire fit')
        plt.plot(thetas, func_s_channel(thetas, *t_s_popt), lw=0.7, c='lime', label='s-channel contribution')
        plt.plot(thetas, func_rest(thetas, *t_s_popt), lw=0.7, c='cyan', label='t-channel and interference')
        #plt.text(-0.9, 2000, textstr, fontsize=10, verticalalignment='top', bbox=props)

        plt.xlabel('polar angle $\mathrm{cos}\,(\\theta)$')
        plt.ylabel('counts')
        plt.grid()
        plt.legend()
        plt.show()

    # determine s-channel event number
    electron_s_channel = np.sum(electron_cos_theta * (t_s_popt[0] * (1 + bin_centers**2)) / ((t_s_popt[0] * (1 + bin_centers**2)) + t_s_popt[1] / (1 - bin_centers)**2))
    # propagate individual bin uncertainty (i.e. Poisson uncertainty)
    electron_s_channel_u_1 = np.sqrt(np.sum((np.sqrt(electron_cos_theta) * (t_s_popt[0] * (1 + bin_centers**2)) / ((t_s_popt[0] * (1 + bin_centers**2)) + t_s_popt[1] / (1 - bin_centers)**2))**2))
    # propagate fit parameter uncertainty/covariance
    t_s_ddA = np.sum(electron_cos_theta * (t_s_popt[1] * (1+bin_centers**2)/((1-bin_centers)**2))/((t_s_popt[0] * (1+bin_centers**2) + t_s_popt[1] * (1+bin_centers**2)/((1-bin_centers)**2))**2))
    t_s_ddB = np.sum(electron_cos_theta * (t_s_popt[0] * (1+bin_centers**2)/((1-bin_centers)**2))/((t_s_popt[0] * (1+bin_centers**2) + t_s_popt[1] * (1+bin_centers**2)/((1-bin_centers)**2))**2))
    electron_s_channel_u_2 = np.sqrt((t_s_ddA * np.sqrt(t_s_pcov[0,0]))**2 + (t_s_ddB * np.sqrt(t_s_pcov[1,1]))**2 + 2 * t_s_ddA * t_s_ddB * t_s_pcov[0,1])
    # combine uncertainties
    electron_s_channel_u = np.sqrt(electron_s_channel_u_1**2 + electron_s_channel_u_2**2)
    
    # print results
    print("fit params:     ", t_s_popt[0], t_s_popt[1])
    print("total ee events:", np.sum(electron_cos_theta), "+-", np.sqrt(np.sum(electron_cos_theta)))
    print("s-channel only: ", electron_s_channel, "+-", electron_s_channel_u)
    
    return electron_s_channel, electron_s_channel_u

In [ ]:
## apply t-s-selection to electron events

# prepare arrays for events (7 different LEP_energies, 4 different event categories)
orig_events = np.zeros((7,4))
orig_events_u = np.zeros((7,4))

# get the seven different LEP-energies
LEP_energies = np.sort(OPAL_data['E_lep'].round(1).unique())

# do for all energy values
for e, energy in enumerate(LEP_energies):
    
    # calculate electron s-channel event numbers
    s_channel_only, s_channel_only_u = perform_t_s_separation(measured_electron[measured_electron['E_lep'].round(1) == energy], bins=20)
    
    # write to event arrays
    orig_events[e][0] = s_channel_only
    orig_events_u[e][0] = s_channel_only_u

# Exercise 3: Measurement of the total production cross sections

For **each** of the seven centre-of-mass energies:
* Determine the number of events in the handronic channel *and* in the three leptonic channels
* Substract the background and correct for selection efficiencies accordingly
* Then, calculate the differnetial cross sections for the hadronic *and* the leptnic channels
* Add the radiation corrections from The table given below. **Don't forget to take the uncertainties (errors) into account!**

| $\sqrt{s}$   \[GeV\]| Correction hadronic channel    \[nb\] |  Correction leptonic channel   \[nb\]|
| --- | --- | --- |
| 88.47 | +2.0  | +0.09 |
| 89.46 | +4.3  | +0.20 |
| 90.22 | +7.7  | +0.36 |
| 91.22 | +10.8 | +0.52 |
| 91.97 | +4.7  | +0.22 |
| 92.96 | -0.2  | -0.01 |
| 93.76 | -1.6  | -0.08 |

Feel free to access these values using the dictionary 'xs_corrections' given below.
* Once the total cross section for all four decay channels at all seven energies have been measured, fit a **Breit-Wigner distribution** to measure the $Z$ boson mass ($m_Z$) and the resonance width ($\Gamma_Z$) and the peak cross section s of the resonance for the hadronic and the leptonic channels. Again, **propagate the uncertainties carefully**.
* Compare your results to the OPAL cross section s and the theoretical predictions. How many degrees of freedom does the fit have? How can you udge if the model is compatible with the measured data? Calculate the  **confidence levels**.
* Calculate the partial widths for all channels from the measured cross sections on the peak. Which is the best partial width to start with? Compare them with the theoretical predictions and the values that you have calculated in the beginning.
* Determine from your results the **number of generations of light neutrinos**. Which assumptions are necessary?
* Discuss in detail the systematic uncertainties in the whole procedure of the analysis. Which assumptions were necessary?

These are some **references** that might be interesting to look up:
* Particle Data Book: https://pdg.lbl.gov/2020/download/Prog.Theor.Exp.Phys.2020.083C01.pdf
** Resonances: https://pdg.lbl.gov/2017/reviews/rpp2017-rev-resonances.pdf
* Precision Electroweak Measurements on the Z Resonance (Combination LEP): https://arxiv.org/abs/hep-ex/0509008
* Measurement of the $Z^0$ mass and width with the OPAL detector at LEP: https://doi.org/10.1016/0370-2693(89)90705-3
* Measurement of the $Z^0$ line shape parameters and the electroweak couplings of charged leptons: https://inspirehep.net/literature/315269
* The OPAL Collaboration, *Precise Determination of the $Z$ Resonance Parameters at LEP: "Zedometry"*: https://arxiv.org/abs/hep-ex/0012018
* Fitting a Breit-Wigner curve using uproot: https://masonproffitt.github.io/uproot-tutorial/07-fitting/index.html

In [ ]:
xs_corrections = pd.DataFrame({ 'energy' : [ 88.47, 89.46, 90.22, 91.22, 91.97, 92.96, 93.76] ,
                              'hadronic' : [2.0, 4.3, 7.7, 10.8, 4.7, -0.2, -1.6],
                              'leptonic' : [0.09, 0.20, 0.36, 0.52, 0.22, -0.01, -0.08]})

## read csv-file of luminosity data
file = 'opal_data/lumi_files/daten_2.csv'
lumi_data = pd.read_csv(file)

# print lumi data
lumi_data

## Calculate cross sections

In [ ]:
## compute the number of measured events for other particles (muon, tau, hadrons)

# do for all event types and all energy values
for p, (particle, dataset) in enumerate(measured_datasets.items()):
    # we already did electrons in the cell above
    if p == 0:
        continue
    
    for e, energy in enumerate(LEP_energies):
        
        # count events and write to event arrays
        orig_events[e][p] = np.sum(dataset['E_lep'].round(1) == energy)
        
        # calculate Poisson uncertainty
        orig_events_u[e][p] = np.sqrt(orig_events[e][p])
    
    
# print data
pd.DataFrame(orig_events, columns = ['e_events', 'm_events', 't_events', 'q_events'])
#pd.DataFrame(orig_events_u, columns = ['e_events_u', 'm_events_u', 't_events_u', 'q_events_u'])

In [ ]:
## correct the measured events using the inverted efficiency matrix

# prepare array
corr_events = np.zeros((7,4))
corr_events_u = np.zeros((7,4))

# do for all energy values
for e in range(7):
    
    # calculate "true" event numbers using inverted efficiency matrix
    corr_events[e] = inverse_means @ orig_events[e]
    
    # propagate uncertainties (cannot be done in form of matrix multiplication because of the exponent 2)
    for p in range(4):
        
        # propagate uncertainties from inverted matrix
        u_correction_sum = np.sum((inverse_errors[p] * orig_events[e])**2)
        
        # propagate Poisson uncertainties
        u_poisson_sum = np.sum((inverse_means[p] * orig_events_u[e])**2)
        
        # combine uncertainties
        corr_events_u[e][p] = np.sqrt(u_correction_sum + u_poisson_sum)
    
        
# print data
pd.DataFrame(corr_events, columns = ['e_events', 'm_events', 't_events', 'q_events'])
#pd.DataFrame(corr_events_u, columns = ['e_events_u', 'm_events_u', 't_events_u', 'q_events_u'])

In [ ]:
## calculate cross sections

# prepare arrays
cross_section = np.zeros((7,4))
cross_section_u = np.zeros((7,4))

# do for all energies
for e in range(7):
    # calculate cross section
    cross_section[e] = corr_events[e] / lumi_data['lumi'][e]
    
    # propagate uncertainties
    for p in range(4):
        cross_section_u[e][p] = cross_section[e][p] * \
            np.sqrt((corr_events_u[e][p]/corr_events[e][p])**2 + (lumi_data['all'][e]/lumi_data['lumi'][e])**2)


## apply radiation corrections
for e in range(7):
    
    # correct lepton data
    cross_section[e][0] += xs_corrections['leptonic'][e]
    cross_section[e][1] += xs_corrections['leptonic'][e]
    cross_section[e][2] += xs_corrections['leptonic'][e]
    
    # correct hadron data
    cross_section[e][3] += xs_corrections['hadronic'][e]
        
# print data
pd.DataFrame(cross_section, columns=['sigma_ee', 'sigma_mm', 'sigma_tt', 'sigma_qq'])
#pd.DataFrame(cross_section_u, columns=['sigma_ee_u', 'sigma_mm_u', 'sigma_tt_u', 'sigma_qq_u'])

In [ ]:
## plot cross sections to energies

plt.figure(figsize=(10, 10))

for p, particle in enumerate(['$\\mathrm{Z}^0\\rightarrow\\mathrm{e}^+\\mathrm{e}^-$',
                             '$\\mathrm{Z}^0\\rightarrow\\mu^+\\mu^-$',
                             '$\\mathrm{Z}^0\\rightarrow\\tau^+\\tau^-$',
                             '$\\mathrm{Z}^0\\rightarrow\\mathrm{q}\\,\\bar{\\mathrm{q}}$']):
    plt.errorbar(2*LEP_energies, cross_section[:,p], yerr=cross_section_u[:,p], fmt="x", label=particle)

plt.xlabel("center-of-mass energy $\\sqrt{s}$ [$\\mathrm{GeV}$]")
plt.ylabel("partial cross section $\\sigma$ [$\\mathrm{nb}$]")
plt.legend()
plt.ylim(0, 2.2)
plt.grid()
plt.show()

## Evaluate cross sections with fit

In [ ]:
## fit Breit-Wigner functions

# define Breit-Wigner function
def breit_wigner(s, mass, gamma, peak):
    return peak * s / (((s - mass**2)**2 / gamma**2) + (s**2 / mass**2))

# define Gaussian function
def gaussian(x, x_0, sigma, a):
    return a * np.exp(-(x-x_0)**2 / (2*sigma**2))


# this function performs the breit wigner fit for the given particle index
def perform_breit_wigner_fits(particle_index, repetitions=int(1e3)):
    particle = particle_index

    LEP_sqrt_s = 2*LEP_energies

    # prepare arrays
    bw_popts = np.zeros((repetitions, 3))
    fit_cross_sections = np.repeat([cross_section[:,particle]], repetitions, axis=0)

    # add normal distributed deviations
    for e in range(7):
        fit_cross_sections[:,e] += np.random.normal(0.0, cross_section_u[e,particle], repetitions)

    # fit a lot of times
    for i in range(repetitions):

        # perform curve fit
        bw_popt, bw_pcov = curve_fit(breit_wigner, LEP_sqrt_s**2, fit_cross_sections[i], p0=[91, 3, 40], maxfev=int(1e6))

        # store results
        bw_popts[i] = bw_popt

        # plot if desired
        energies = np.linspace(np.amin(LEP_sqrt_s)-.5, np.amax(LEP_sqrt_s)+.5, int(1e4))
        plt.plot(energies, breit_wigner(energies**2, *bw_popt), c="#888888")

    # plot original cross sections on top of fits
    plt.errorbar(LEP_sqrt_s, cross_section[:,particle], yerr=cross_section_u[:,particle], fmt="x")
    plt.show()


    # calculate mean fit parameters
    bw_popt = np.mean(bw_popts, axis=0)
    
    # calculate covariances
    bw_pcov = np.zeros((3,3))
    for i in range(3):
        for j in range(i, 3):
            bw_pcov[i][j] = bw_pcov[j][i] = np.sum((bw_popts[:,i]-bw_popt[i]) * (bw_popts[:,j]-bw_popt[j]))/(repetitions-1)

    # print results
    print(bw_popt[0], "+-", np.sqrt(bw_pcov[0,0]))
    print(bw_popt[1], "+-", np.sqrt(bw_pcov[1,1]))
    print(bw_popt[2], "+-", np.sqrt(bw_pcov[2,2]))

            
    # return fit params and covariances
    return bw_popt, bw_pcov



# fit electron events
ee_popt, ee_pcov = perform_breit_wigner_fits(0)

# fit muon events
mm_popt, mm_pcov = perform_breit_wigner_fits(1)

# fit tau events
tt_popt, tt_pcov = perform_breit_wigner_fits(2)

# fit hadron events
qq_popt, qq_pcov = perform_breit_wigner_fits(3)


In [ ]:
## determine partial decay width for electron-positron events

# unit factor nb^(1/2) to MeV^(-1)
unit_factor = np.sqrt(2.56819)
# unit factor nb to GeV^(-2)
unit_factor2 = 2.56819e-6
# unit factor GeV^(2)/MeV to MeV
unit_factor3 = 1e6

# do calculation
partial_ee = unit_factor * np.sqrt(ee_popt[2] / (12*np.pi)) * ee_popt[0] * ee_popt[1]

# propagate uncertainties and covariances
ddPeak = ee_popt[0] * ee_popt[1] / (2 * np.sqrt(12 * np.pi * ee_popt[2]))
ddMass = ee_popt[1] * np.sqrt(ee_popt[2] / (12*np.pi))
ddGamma = ee_popt[0] * np.sqrt(ee_popt[2] / (12*np.pi))
partial_ee_u = unit_factor * np.sqrt(ddMass**2 * ee_pcov[0,0] + ddGamma**2 * ee_pcov[1,1] + ddPeak**2 * ee_pcov[2,2] + \
                       2 * ddMass * ddGamma * ee_pcov[0,1] + 2 * ddMass * ddPeak * ee_pcov[0,2] + \
                       2 * ddGamma * ddPeak * ee_pcov[1,2])
# print result
print("Gamma_ee: ", partial_ee, "+-", partial_ee_u, "MeV")



## determine partial decay width for muon events
partial_mm = unit_factor2 * unit_factor3 * mm_popt[0]**2 * mm_popt[1]**2 * mm_popt[2] / (12 * np.pi * partial_ee)

# propagate uncertainties and covariances
ddPeak = mm_popt[0]**2 * mm_popt[1]**2 / (12*np.pi * partial_ee)
ddMass = mm_popt[0] * mm_popt[1]**2 * mm_popt[2] / (6*np.pi * partial_ee)
ddGamma = mm_popt[0]**2 * mm_popt[1] * mm_popt[2] / (6*np.pi * partial_ee)
ddPartial_ee = -mm_popt[2] * mm_popt[0]**2 * mm_popt[1]**2 / (12*np.pi * partial_ee**2)
partial_mm_u = unit_factor2 * unit_factor3 * np.sqrt(ddMass**2 * mm_pcov[0,0] + ddGamma**2 * mm_pcov[1,1] + \
                                                     ddPeak**2 * mm_pcov[2,2] + 2 * ddMass * ddGamma * mm_pcov[0,1] + \
                                                     2 * ddMass * ddPeak * mm_pcov[0,2] + 2 * ddGamma * ddPeak * mm_pcov[1,2] + \
                                                     ddPartial_ee**2 * partial_ee_u**2)
# print result
print("Gamma_mm: ", partial_mm, "+-", partial_mm_u, "MeV")



## determine partial decay width for tau events
partial_tt = unit_factor2 * unit_factor3 * tt_popt[0]**2 * tt_popt[1]**2 * tt_popt[2] / (12 * np.pi * partial_ee)

# propagate uncertainties and covariances
ddPeak = tt_popt[0]**2 * tt_popt[1]**2 / (12*np.pi * partial_ee)
ddMass = tt_popt[0] * tt_popt[1]**2 * tt_popt[2] / (6*np.pi * partial_ee)
ddGamma = tt_popt[0]**2 * tt_popt[1] * tt_popt[2] / (6*np.pi * partial_ee)
ddPartial_ee = -tt_popt[2] * tt_popt[0]**2 * tt_popt[1]**2 / (12*np.pi * partial_ee**2)
partial_tt_u = unit_factor2 * unit_factor3 * np.sqrt(ddMass**2 * tt_pcov[0,0] + ddGamma**2 * tt_pcov[1,1] + \
                                                     ddPeak**2 * tt_pcov[2,2] + 2 * ddMass * ddGamma * tt_pcov[0,1] + \
                                                     2 * ddMass * ddPeak * tt_pcov[0,2] + 2 * ddGamma * ddPeak * tt_pcov[1,2] + \
                                                     ddPartial_ee**2 * partial_ee_u**2)
# print result
print("Gamma_tt: ", partial_tt, "+-", partial_tt_u, "MeV")



## determine partial decay width for hadron events
partial_qq = unit_factor2 * unit_factor3 * qq_popt[0]**2 * qq_popt[1]**2 * qq_popt[2] / (12 * np.pi * partial_ee)

# propagate uncertainties and covariances
ddPeak = qq_popt[0]**2 * qq_popt[1]**2 / (12*np.pi * partial_ee)
ddMass = qq_popt[0] * qq_popt[1]**2 * qq_popt[2] / (6*np.pi * partial_ee)
ddGamma = qq_popt[0]**2 * qq_popt[1] * qq_popt[2] / (6*np.pi * partial_ee)
ddPartial_ee = -qq_popt[2] * qq_popt[0]**2 * qq_popt[1]**2 / (12*np.pi * partial_ee**2)
partial_qq_u = unit_factor2 * unit_factor3 * np.sqrt(ddMass**2 * qq_pcov[0,0] + ddGamma**2 * qq_pcov[1,1] + \
                                                     ddPeak**2 * qq_pcov[2,2] + 2 * ddMass * ddGamma * qq_pcov[0,1] + \
                                                     2 * ddMass * ddPeak * qq_pcov[0,2] + 2 * ddGamma * ddPeak * qq_pcov[1,2] + \
                                                     ddPartial_ee**2 * partial_ee_u**2)
# print result
print("Gamma_qq: ", partial_qq, "+-", partial_qq_u, "MeV")

In [ ]:
## calculate breit wigner fit mean values

# calculate z0 boson mass via weighted average
z0_mass = (ee_popt[0]/ee_pcov[0,0] + mm_popt[0]/mm_pcov[0,0] + tt_popt[0]/tt_pcov[0,0] + qq_popt[0]/qq_pcov[0,0]) / \
            (1/ee_pcov[0,0] + 1/mm_pcov[0,0] + 1/tt_pcov[0,0] + 1/qq_pcov[0,0])
# propagate uncertainty
z0_mass_u = 1 / np.sqrt(1/ee_pcov[0,0] + 1/mm_pcov[0,0] + 1/tt_pcov[0,0] + 1/qq_pcov[0,0])

# print result
print("z0 mass:        ", z0_mass, "+-", z0_mass_u, "GeV")



# calculate z0 boson total decay width via weighted average
z0_decay_width = (ee_popt[1]/ee_pcov[1,1] + mm_popt[1]/mm_pcov[1,1] + tt_popt[1]/tt_pcov[1,1] + qq_popt[1]/qq_pcov[1,1]) / \
            (1/ee_pcov[1,1] + 1/mm_pcov[1,1] + 1/tt_pcov[1,1] + 1/qq_pcov[1,1])
# propagate uncertainty
z0_decay_width_u = 1 / np.sqrt(1/ee_pcov[1,1] + 1/mm_pcov[1,1] + 1/tt_pcov[1,1] + 1/qq_pcov[1,1])

# print result
print("z0 decay width: ", z0_decay_width, "+-", z0_decay_width_u, "GeV")

# Exercise 4: Forward-backward asymmetry and $\sin^2(\theta_\text{W})$ in muon final states

* Using the **muon channel only**, measure the forward-backward asymmetry $\mathcal{A}_\text{FB}$ using OPAL data and muon Monte Carlo events. Take into account the radiation corrections given below. 

| $\sqrt{s}$   \[GeV\]| Radiation correction [-]|  
| --- | --- | 
| 88.47 | 0.021512  | 
| 89.46 | 0.019262  | 
| 90.22 | 0.016713  | 
| 91.22 | 0.018293  | 
| 91.97 | 0.030286  | 
| 92.96 | 0.062196  | 
| 93.76 | 0.093850  | 

Feel free to use the dictionary 'radiation_corrections' given below.

* Measure the **Weinberg angle** as $\sin^2(\theta_\text{W})$. Compare the measurement with the literature.

In [ ]:
radiation_corrections = pd.DataFrame({ 'energy' : [ 88.47, 89.46, 90.22, 91.22, 91.97, 92.96, 93.76] ,
                                'correction' : [0.021512, 0.019262, 0.016713, 0.018293, 0.030286, 0.062196, 0.093850]})

First, we perform the calculation of the weinberg angle for the monte carlo dataset:

In [ ]:
# calculate the counts forwards and backwards
forward_sim = sum(simulated_muon['cos_thet'] > 0)
backward_sim = sum(simulated_muon['cos_thet'] < 0)

# uncertainties according to poission
f_sim_u = np.sqrt(forward_sim)
b_sim_u = np.sqrt(backward_sim)

# calculate forward-backward assym. and uncertainty
AFB_sim = (forward_sim - backward_sim) / (forward_sim + backward_sim) + radiation_corrections['correction'][3]
AFB_sim_u = np.sqrt( ((2*backward_sim)/(forward_sim + backward_sim)**2 * f_sim_u)**2 
                    + ((2*forward_sim)/(forward_sim + backward_sim)**2 * b_sim_u)**2)

# calculate weinberg angle and uncertainty
sinsq_thetaW_sim = 0.25 - np.sqrt(AFB_sim/3) / 4
sinsq_thetaW_sim_u = 1 / (8 * np.sqrt(3) * np.sqrt(AFB_sim)) * AFB_sim_u

print('For MC muons: \n \t \t sin²(theta) = ' + 
      str(np.round(sinsq_thetaW_sim, 4)) + ' +- ' +str(np.round(sinsq_thetaW_sim_u, 4)))

We then do the same for all the energies within the measured dataset:

In [ ]:
# setup arrays to store the results
AFB_meas = np.zeros(7)
AFB_meas_u = np.zeros(7)
sinsq_thetaW_meas = np.zeros(7)
sinsq_thetaW_meas_u = np.zeros(7)

# do the recalculation for each LEP energy
for e, energy in enumerate(LEP_energies):
    
    # extract only the events for that energy
    dataset = measured_muon[measured_muon['E_lep'].round(1) == energy]

    # calculate the counts forwards and backwards
    forward_meas = sum(dataset['cos_thet'] > 0)
    backward_meas = sum(dataset['cos_thet'] < 0)

    # uncertainties according to poission
    f_meas_u = np.sqrt(forward_meas)
    b_meas_u = np.sqrt(backward_meas)

    # calculate forward-backward assym. and uncertainty
    AFB_meas[e] = (forward_meas - backward_meas) / (forward_meas + backward_meas) + radiation_corrections['correction'][e]
    AFB_meas_u[e] = np.sqrt( ((2*backward_meas)/(forward_meas + backward_meas)**2 * f_meas_u)**2 
                    + ((2*forward_meas)/(forward_meas + backward_meas)**2 * b_meas_u)**2)
    
    # calculate weinberg angle and uncertainty
    sinsq_thetaW_meas[e] = 0.25 - np.sqrt(AFB_meas[e]/3) / 4
    sinsq_thetaW_meas_u[e] = 1 / (8 * np.sqrt(3) * np.sqrt(AFB_meas[e])) * AFB_meas_u[e]
    
    if e==4:   # here, the approximation is exact
        print('For measured muons at sqrt(s) = ' + str(radiation_corrections['energy'][e]) + ': \n \t \t sin²(theta) = ' + 
              str(np.round(sinsq_thetaW_meas[e], 4)) + ' +- ' +str(np.round(sinsq_thetaW_meas_u[e], 4)))
    
plt.errorbar(radiation_corrections['energy'], AFB_meas, AFB_meas_u, fmt='x', ecolor='r')
plt.xlabel("center-of-mass energy $\\sqrt{s}$ [$\\mathrm{GeV}$]")
plt.ylabel("forward backward assymetry $A_\\mathrm{FB}$")
plt.grid()
plt.show()

P.S: For the `nan`-values, $A_\mathrm{FB}$ was negative, so the square root failed.

# Exercise 5: Tests on lepton universality¶

* Test the lepton universality from the total cross sectinos on the peak for $Z\to e^+ e^-$, $Z\to \mu^+ \mu^-$ and $Z\to \tau^+ \tau^-$ events. What is the ratio of the total cross section of the hadronic channel to the leptonic channels on the peak? Compare with the ratios obtained from the branching rations and discuss possible differences.

In [ ]:
## calculate ratios between leptonic and hadronic cross section

# prepare arrays
lepton_to_hadron_ratios = np.zeros((7,3))
lepton_to_hadron_ratios_u = np.zeros((7,3))

# do for all energies
for e in range(7):
    # devide lepton cross sections by hadronic cross section
    lepton_to_hadron_ratios[e] = cross_section[e,:-1] / cross_section[e,-1]
    
    # propagate uncertainties
    for p in range(3):
        lepton_to_hadron_ratios_u[e][p] = lepton_to_hadron_ratios[e][p] * \
            np.sqrt((cross_section_u[e][p]/cross_section[e][p])**2 + (cross_section_u[e][-1]/cross_section[e][-1])**2)


## plot results
for p, particle in enumerate(['$\\sigma^\\mathrm{e}/\\sigma^\\mathrm{q}$', '$\\sigma^\\mu/\\sigma^\\mathrm{q}$', '$\\sigma^\\tau/\\sigma^\\mathrm{q}$']):
    plt.errorbar(2*LEP_energies, lepton_to_hadron_ratios[:,p], yerr=lepton_to_hadron_ratios_u[:,p], fmt="x", label=particle)

#plt.ylim(0.03, 0.06)
plt.axhline(0.052716, label="theory", c="r")
plt.legend()
plt.xlabel("center-of-mass energy $\\sqrt{s}$ [$\\mathrm{GeV}$]")
plt.ylabel("ratio of cross sections [$1$]")
plt.grid()
plt.show()

In [ ]:
## calculate number of light neutrinos

# calculate invisible decay width
inv_decay_width = z0_decay_width*1e3 - partial_ee \
                    - unit_factor2 * unit_factor3 * mm_popt[0]**2 * mm_popt[1]**2 * mm_popt[2] / (12 * np.pi * partial_ee) \
                    - unit_factor2 * unit_factor3 * tt_popt[0]**2 * tt_popt[1]**2 * tt_popt[2] / (12 * np.pi * partial_ee) \
                    - unit_factor2 * unit_factor3 * qq_popt[0]**2 * qq_popt[1]**2 * qq_popt[2] / (12 * np.pi * partial_ee)
# propagate uncertainties and covariances
# calculate partial derivatives
ddPartial_ee = -1 + unit_factor2 * unit_factor3 * mm_popt[0]**2 * mm_popt[1]**2 * mm_popt[2] / (12 * np.pi * partial_ee**2) \
                  + unit_factor2 * unit_factor3 * tt_popt[0]**2 * tt_popt[1]**2 * tt_popt[2] / (12 * np.pi * partial_ee**2) \
                  + unit_factor2 * unit_factor3 * qq_popt[0]**2 * qq_popt[1]**2 * qq_popt[2] / (12 * np.pi * partial_ee**2)
ddPeak_m = mm_popt[0]**2 * mm_popt[1]**2 / (12*np.pi * partial_ee)
ddMass_m = mm_popt[0] * mm_popt[1]**2 * mm_popt[2] / (6*np.pi * partial_ee)
ddGamma_m = mm_popt[0]**2 * mm_popt[1] * mm_popt[2] / (6*np.pi * partial_ee)
ddPeak_t = tt_popt[0]**2 * tt_popt[1]**2 / (12*np.pi * partial_ee)
ddMass_t = tt_popt[0] * tt_popt[1]**2 * tt_popt[2] / (6*np.pi * partial_ee)
ddGamma_t = tt_popt[0]**2 * tt_popt[1] * tt_popt[2] / (6*np.pi * partial_ee)
ddPeak_q = qq_popt[0]**2 * qq_popt[1]**2 / (12*np.pi * partial_ee)
ddMass_q = qq_popt[0] * qq_popt[1]**2 * qq_popt[2] / (6*np.pi * partial_ee)
ddGamma_q = qq_popt[0]**2 * qq_popt[1] * qq_popt[2] / (6*np.pi * partial_ee)
# sum all uncertainties
inv_decay_width_u = np.sqrt((z0_decay_width_u*1e3)**2 + (ddPartial_ee * partial_ee_u)**2 + \
                            ddPeak_m**2 * mm_pcov[2,2] + ddMass_m**2 * mm_pcov[0,0] + ddGamma_m**2 * mm_pcov[1,1] + \
                            ddPeak_t**2 * tt_pcov[2,2] + ddMass_t**2 * tt_pcov[0,0] + ddGamma_t**2 * tt_pcov[1,1] + \
                            ddPeak_q**2 * qq_pcov[2,2] + ddMass_q**2 * qq_pcov[0,0] + ddGamma_q**2 * qq_pcov[1,1])

# divide by theoretical value per neutrino type
number_neutrinos = inv_decay_width / 165.9
number_neutrinos_u = inv_decay_width_u / 165.9

print("Number of light neutrinos: ", number_neutrinos, "+-", number_neutrinos_u)
print("Theory: 3")